<a href="https://colab.research.google.com/github/Luiz-Henrique05/Recomendacao-de-filmes/blob/main/Recomenda%C3%A7%C3%A3o_de_Filmes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1. Análise e manipulação dos dados

In [ ]:
url_filmes = 'https://drive.google.com/uc?id=17kchscPu9hURmhHw7ZN5ir2O_-ObZdi5'
filmes = pd.read_csv(url_filmes, sep=',') # Filmes obtidos pela base do MovieLens

url_ratings = 'https://drive.google.com/uc?id=17mOz_4CUb_Xw_uAIlgLf6D7cJY5q9747'
ratings = pd.read_csv(url_ratings, sep=';') # Avaliações obtidas pela base do MovieLens

In [ ]:
#filmes = pd.read_csv('/content/drive/My Drive/Projeto Filmes/Filmes.csv', sep=',') # Filmes obtidos pela base do MovieLens
#ratings = pd.read_csv('/content/drive/My Drive/Projeto Filmes/Ratings.csv', sep=';') # Avaliações obtidas pela base do MovieLens

In [ ]:
filmes.head() # Código do filme / Título / Gêneros / Ano de lançamento

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [ ]:
ratings.head() # Código do usuário / Código do filme / Nota da avaliação

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [ ]:
filmes.shape # Mais de 9600 filmes na base

(9737, 4)

In [ ]:
ratings.shape # Mais de 100 mil avaliações

(100836, 3)

In [ ]:
df = filmes.merge(ratings, on='movieId') # Agrupar dados --> Cada nota dada pelo usuário com os filmes
df.head()

,movieId,title,genres,year,userId,rating
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,1,4.0
1,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,5,4.0
2,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,7,4.5
3,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,15,2.5
4,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,17,4.5


#### 3. - Filtragem Colaborativa -
Esse método faz recomendações com base em padrões de comportamento entre usuários. Ele identifica grupos de usuários com **preferências semelhantes** e usa essas semelhanças para sugerir filmes. Por exemplo, se muitos usuários que têm **gostos parecidos** com o usuário atual **assistiram e gostaram de determinados filmes**, esses filmes podem ser **recomendados**. Assim, indica que são filmes que agradam o mesmo público, indentificando nichos. Esse método é eficaz para captar tendências coletivas, mas depende de um volume grande de dados de diversos usuários.

In [ ]:
tabela_filmes = pd.pivot_table(df, index='title', columns='userId', values='rating').fillna(0) # Notas de cada usuário vinculadas aos filmes / OBS: A maioria das pessoas não viram todos os filmes
tabela_filmes.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import sklearn.metrics.pairwise as pw # Similaridade por Coseno --> Técnica fundamental para as recomendações

--> Transforma **filmes em vetores**, em um plano cartesiano, e os valores das **avaliações dos usuários é o que determina suas coordenadas**. Portanto, pelo cálculo do coseno desses ângulos é possível encontrar similaridades: Vetores com coordenadas parecidas representam filmes cujos mesmos **usuários deram notas parecidas**, indicando que agradam o mesmo tipo de público

In [ ]:
rec = pw.cosine_similarity(tabela_filmes)
rec_df = pd.DataFrame(rec, columns=tabela_filmes.index, index=tabela_filmes.index)
rec_df.head()

title,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,(500) Days of Summer,*batteries not included,...,Zulu,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
title,,,,,,,,,,,,,,,,,,,,,
'71,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation,0.0,1.000000,0.707107,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight,0.0,0.707107,1.000000,0.000000,0.000000,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Salem's Lot,0.0,0.000000,0.000000,1.000000,0.857493,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Til There Was You,0.0,0.000000,0.000000,0.857493,1.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


In [ ]:
cossine_df = pd.DataFrame(rec_df['Lord of the Rings, The'].sort_values(ascending=False)) # Madagascar
cossine_df.columns = ['Recomendações']
cossine_df.head(20)

,Recomendações
title,
"Lord of the Rings, The",1.000000
Excalibur,0.542994
Transformers: The Movie,0.527281
Six-String Samurai,0.486484
Blackbeard's Ghost,0.485284
Young Sherlock Holmes,0.468331
Ladyhawke,0.458975
"Clan of the Cave Bear, The",0.446785
"Newton Boys, The",0.445865


In [ ]:
cossine_df = pd.DataFrame(rec_df['Lion King, The'].sort_values(ascending=False)) # Rei Leão
cossine_df.columns = ['Recomendações']
cossine_df.head(20)

,Recomendações
title,
"Lion King, The",1.000000
Aladdin,0.717909
Beauty and the Beast,0.710421
Mrs. Doubtfire,0.650090
"Mask, The",0.638915
Jurassic Park,0.614847
Jumanji,0.588438
Forrest Gump,0.586446
Babe,0.576214


In [ ]:
cossine_df = pd.DataFrame(rec_df['Blade Runner'].sort_values(ascending=False)) # Blade Runner
cossine_df.columns = ['Recomendações']
cossine_df.head(20)

,Recomendações
title,
Blade Runner,1.000000
2001: A Space Odyssey,0.670736
Alien,0.607595
"Terminator, The",0.588224
Monty Python and the Holy Grail,0.583766
Star Wars: Episode V - The Empire Strikes Back,0.582449
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb,0.580435
Apocalypse Now,0.579697
Aliens,0.572017


In [ ]:
cossine_df = pd.DataFrame(rec_df['Iron Man'].sort_values(ascending=False)) # Homem de Ferro
cossine_df.columns = ['Recomendações']
cossine_df.head(20)

,Recomendações
title,
Iron Man,1.000000
"Dark Knight, The",0.670088
WALL·E,0.666213
"Avengers, The",0.638990
Iron Man 2,0.636235
Avatar,0.628082
Batman Begins,0.600719
Up,0.593971
Star Trek,0.586045


In [ ]:
cossine_df = pd.DataFrame(rec_df['Green Mile, The'].sort_values(ascending=False)) # À Espera de um Milagre
cossine_df.columns = ['Recomendações']
cossine_df.head(20)

,Recomendações
title,
"Green Mile, The",1.000000
Saving Private Ryan,0.561304
"Sixth Sense, The",0.549818
Catch Me If You Can,0.532505
"Matrix, The",0.518537
American History X,0.516832
Fight Club,0.516003
"Beautiful Mind, A",0.506656
"Shawshank Redemption, The",0.497148


#### 3. - Filtragem Baseada em Conteúdo -
Nessa abordagem, o foco está nas características dos filmes que o próprio usuário já demonstrou interesse, como **gênero, diretor, atores ou temas específicos**. Com base nesses atributos, o sistema sugere filmes com **características semelhantes**, como similaridades pelo tema ou sinopse. Esse método é especialmente útil para personalizar recomendações quando o histórico de um único usuário é a principal referência, sem depender das preferências de outros.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:

url_tags = 'https://drive.google.com/uc?id=17owvLLXuJf_xpbwD8idu-Tg_QB9Kx4C9'
url_dados = 'https://drive.google.com/uc?id=17jcUjy2Coca0pNSdm_0DvrUpTQyR52sx'

tags = pd.read_csv(url_tags) # Tags obtidas pela base do MovieLens
dados = pd.read_csv(url_dados) # Filmes obtidos pela base do IMDB

In [ ]:
dados.head() # Ficha técnica dos filmes

,Name,Year,Stars,Score,Time,Votes,Total,Tags,Directors_Cast,Discription
0,Pulp Fiction,1994,8.9,94.0,154,"1,871,051",$107.93,\nCrime Drama,Quentin Tarantino John Travolta Uma Thurman Sa...,\nThe lives of two mob hitmen a boxer a gangst...
1,The Amazing Spider-Man 2,2014,6.6,53.0,142,"425,529",$202.85,\nAction Adventure Fantasy,Marc Webb Andrew Garfield Emma Stone Jamie Fox...,\nWhen New York is put under siege by Oscorp i...
2,The Shawshank Redemption,1994,9.3,80.0,142,"2,409,436",$28.34,\nDrama,Frank Darabont Tim Robbins Morgan Freeman Bob ...,\nTwo imprisoned men bond over a number of yea...
3,Star Wars: Episode IV - A New Hope,1977,8.6,90.0,121,"1,255,464",$322.74,\nAction Adventure Fantasy,George Lucas Mark Hamill Harrison Ford Carrie ...,\nLuke Skywalker joins forces with a Jedi Knig...
4,Back to the Future,1985,8.5,87.0,116,"1,087,878",$210.61,\nAdventure Comedy Sci-Fi,Robert Zemeckis Michael J. Fox Christopher Llo...,\nMarty McFly a 17-year-old high school studen...


In [ ]:
tags.head()

,movieId,tag
0,1,"Owned,imdb top 250,Pixar,Pixar,time travel,chi..."
1,2,"Robin Williams,time travel,fantasy,based on ch..."
2,3,"funny,best friend,duringcreditsstinger,fishing..."
3,4,"based on novel or book,chick flick,divorce,int..."
4,5,"aging,baby,confidence,contraception,daughter,g..."


In [ ]:
dados.shape # Quase 10 mil filmes

(9937, 10)

In [ ]:
tags.shape # Mais de 45 mil tags fornecidas pelos usuários

(45256, 2)

In [ ]:
filmes['movieId'] = filmes['movieId'].apply(lambda x: str(x))

In [ ]:
df2 = filmes.merge(dados, left_on='title', right_on='Name', how='left')
df2 = df2.merge(tags, left_on='movieId', right_on='movieId', how='left')
df2['Infos'] = df2['genres'] + str(df2['Directors_Cast']) + str(df2['Discription']) + df2['tag']
df2.head()

,movieId,title,genres,year,Name,Year,Stars,Score,Time,Votes,Total,Tags,Directors_Cast,Discription,tag,Infos
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,Toy Story,1995,8.3,95.0,81,"908,794",$191.80,\nAnimation Adventure Comedy,John Lasseter Tom Hanks Tim Allen Don Rickles ...,\nA cowboy doll is profoundly threatened and j...,"Owned,imdb top 250,Pixar,Pixar,time travel,chi...",Adventure|Animation|Children|Comedy|Fantasy0 ...
1,2,Jumanji,Adventure|Children|Fantasy,1995,Jumanji,1995,7.0,39.0,104,"316,485",$100.48,\nAdventure Comedy Family,Joe Johnston Robin Williams Kirsten Dunst Bonn...,\nWhen two kids find and play a magical board ...,"Robin Williams,time travel,fantasy,based on ch...",Adventure|Children|Fantasy0 John Lasset...
2,3,Grumpier Old Men,Comedy|Romance,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"funny,best friend,duringcreditsstinger,fishing...",Comedy|Romance0 John Lasseter Tom Hanks...
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995,Waiting to Exhale,1995,6.0,NaN,124,"9,911",$67.05,\nComedy Drama Romance,Forest Whitaker Whitney Houston Angela Bassett...,NaN,"based on novel or book,chick flick,divorce,int...",Comedy|Drama|Romance0 John Lasseter Tom...
4,5,Father of the Bride Part II,Comedy,1995,Father of the Bride Part II,1995,6.1,49.0,106,"35,472",$76.59,\nComedy Family Romance,Charles Shyer Steve Martin Diane Keaton Martin...,\nGeorge Banks must deal not only with the pre...,"aging,baby,confidence,contraception,daughter,g...",Comedy0 John Lasseter Tom Hanks Tim All...


In [ ]:
vec = TfidfVectorizer()  # Processamento de Linguagem Natural --> Técnica fundamental para as recomendações
tfidf = vec.fit_transform(df2['Infos'].apply(lambda x: np.str_(x)))

--> Utiliza técnicas de Processamento de Linguagem Natural (NLP) para identificar a similaridade entre filmes. Para isso, aplica o método **TF-IDF (Term Frequency-Inverse Document Frequency)**, que ajuda a analisar a **relevância de palavras** por pesos em textos específicos, extraindo informações importantes de atributos como tags, diretor, elenco e sinopse dos filmes. A partir desses dados, calcula a similaridade entre filmes usando Cosine Similarity, uma métrica que mede o ângulo entre vetores (no caso, representações textuais dos filmes). Isso permitirá recomendar **filmes que compartilham características semelhantes** aos que o usuário já demonstrou interesse

In [ ]:
sim = cosine_similarity(tfidf)
sim

array([[1.        , 0.08465854, 0.01312877, ..., 0.01361825, 0.02186222,
        0.01361825],
       [0.08465854, 1.        , 0.02747468, ..., 0.0599022 , 0.0949832 ,
        0.0599022 ],
       [0.01312877, 0.02747468, 1.        , ..., 0.19453706, 0.21472584,
        0.19453706],
       ...,
       [0.01361825, 0.0599022 , 0.19453706, ..., 1.        , 0.68030435,
        1.        ],
       [0.02186222, 0.0949832 , 0.21472584, ..., 0.68030435, 1.        ,
        0.68030435],
       [0.01361825, 0.0599022 , 0.19453706, ..., 1.        , 0.68030435,
        1.        ]])

In [ ]:
sim_df2 = pd.DataFrame(sim, columns=df2['title'], index=df2['title'])
sim_df2.head()

title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Heat,Sabrina,Tom and Huck,Sudden Death,...,Gintama: The Movie,anohana: The Flower We Saw That Day - The Movie,Silver Spoon,Love Live! The School Idol Movie,Jon Stewart Has Left the Building,Black Butler: Book of the Atlantic,No Game No Life: Zero,Flint,Bungo Stray Dogs: Dead Apple,Andrew Dice Clay: Dice Rules
title,,,,,,,,,,,,,,,,,,,,,
Toy Story,1.000000,0.084659,0.013129,0.016002,0.033845,0.007786,0.007786,0.006416,0.052864,0.007285,...,0.099869,0.013618,0.013618,0.013618,0.013618,0.013618,0.013618,0.013618,0.021862,0.013618
Jumanji,0.084659,1.000000,0.027475,0.151885,0.103882,0.003845,0.003845,0.052921,0.087719,0.041849,...,0.076600,0.059902,0.059902,0.059902,0.059902,0.059902,0.059902,0.059902,0.094983,0.059902
Grumpier Old Men,0.013129,0.027475,1.000000,0.139910,0.196165,0.046368,0.046368,0.043952,0.131067,0.072365,...,0.179095,0.194537,0.194537,0.194537,0.194537,0.194537,0.194537,0.194537,0.214726,0.194537
Waiting to Exhale,0.016002,0.151885,0.139910,1.000000,0.278759,0.019550,0.019550,0.255034,0.314388,0.151062,...,0.373854,0.407665,0.407665,0.407665,0.407665,0.407665,0.407665,0.407665,0.448173,0.407665
Father of the Bride Part II,0.033845,0.103882,0.196165,0.278759,1.000000,0.011208,0.011208,0.081509,0.261189,0.144008,...,0.356444,0.372773,0.372773,0.372773,0.372773,0.372773,0.372773,0.372773,0.427902,0.372773


In [ ]:
final_df = pd.DataFrame(sim_df2['X-Men'].sort_values(ascending=False)) # X-Men
final_df.columns = ['Recomendações']
final_df.head(20)

,Recomendações
title,
X-Men,1.000000
X2: X-Men United,0.932362
X-Men: The Last Stand,0.780022
X-Men Origins: Wolverine,0.721653
Spider-Man 2,0.695068
Spider-Man 3,0.643332
Spider-Man,0.631787
Superman,0.626561
Spawn,0.609345


In [ ]:
final_df = pd.DataFrame(sim_df2['The Conjuring 2'].sort_values(ascending=False)) # Invocação do Mal 2
final_df.columns = ['Recomendações']
final_df.head(20)

,Recomendações
title,
The Conjuring 2,1.000000
"Amityville Horror, The",0.549718
Insidious: The Last Key,0.498562
"Haunting, The",0.473673
"Haunted House 2, A",0.471262
House on Haunted Hill,0.471206
"Conjuring, The",0.420736
Annabelle,0.418973
Insidious,0.398433


In [ ]:
final_df = pd.DataFrame(sim_df2['Mission: Impossible'].sort_values(ascending=False)) # Missão Impossível
final_df.columns = ['Recomendações']
final_df.head(20)

,Recomendações
title,
Mission: Impossible,1.000000
Mission: Impossible - Rogue Nation,0.621930
Mission: Impossible II,0.607695
Mission: Impossible III,0.545700
Mission: Impossible - Ghost Protocol,0.498338
Spy Game,0.388894
Jack Reacher,0.387798
Knight and Day,0.374968
Tinker Tailor Soldier Spy,0.354046


In [ ]:
final_df = pd.DataFrame(sim_df2[''].sort_values(ascending=False)) # Liga da Justiça
final_df.columns = ['Recomendações']
final_df.head(20)

,Recomendações
title,
"Lord of the Rings, The",1.000000
The Hobbit: The Battle of the Five Armies,0.584850
"Lord of the Rings: The Fellowship of the Ring, The",0.554986
"Lord of the Rings: The Two Towers, The",0.550827
"Lord of the Rings: The Return of the King, The",0.539814
"Hobbit: The Desolation of Smaug, The",0.533540
"Hobbit: An Unexpected Journey, The",0.466280
"Chronicles of Narnia: The Lion, the Witch and the Wardrobe, The",0.270709
Goosebumps,0.270250


In [ ]:
final_df = pd.DataFrame(sim_df2['Back to the Future'].sort_values(ascending=False)) # De volta para o Futuro
final_df.columns = ['Recomendações']
final_df.head(20)

,Recomendações
title,
Back to the Future,1.000000
Back to the Future Part II,0.860040
Back to the Future Part III,0.761418
"Time Machine, The",0.664804
Frequency,0.629356
Peggy Sue Got Married,0.599153
Bill & Ted's Excellent Adventure,0.598600
Somewhere in Time,0.577223
Bill & Ted's Bogus Journey,0.569298
